In [8]:
import numpy as np
import pysindy as ps
from pysindy import SINDy
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from pysindy.optimizers import STLSQ


Loading the data from the Kaggle dataset

In [9]:
data = np.genfromtxt('/home/luka/Desktop/projekti/sindy_tutorial/archive/reconstruction/train.csv', delimiter=',', skip_header=1)

Extracting the variables from the dataset

In [10]:
t = data[:, 0]
X = data[:, 1:3]

#time step
dt = t[1] - t[0]

Let's see what SINDy identifies by default. We use the STSLQ as an L1 regularization in the sparse regression.

In [11]:
model1 = SINDy(optimizer=STLSQ(threshold=0.1))

# Naming the variables
model1.fit(X, t=dt, feature_names=["x", "v"])

model1.print()
print("Model score (R^2):", model1.score(X, t=dt))

(x)' =  0.658 v + -0.314 x^2 +  0.254 x v
(v)' = -0.758 x + -0.250 x^2 +  0.186 x v
Model score (R^2): 0.9395700172025041


By default, SINDy uses the polinomial library. 

In [12]:
feature_names = model1.get_feature_names()
print("Functions in the Theta library:", feature_names)

Functions in the Theta library: ['1', 'x', 'v', 'x^2', 'x v', 'v^2']


Now, let's modify the Theta library (containing basis functions). Alongside polinomials, we will add sin/cos basis (Fourie library). Then, we will see if the identified system is changed:

In [13]:
from pysindy.feature_library import PolynomialLibrary, FourierLibrary

# Joining the Polynomial and Fourier library
combined_lib = PolynomialLibrary(degree=2) + FourierLibrary(n_frequencies=1)

model2 = SINDy(feature_library=combined_lib,optimizer=STLSQ(threshold=0.1))
model2.fit(X, t=dt, feature_names=["x", "v"])

model2.print()
print("Model score (R^2):", model2.score(X, t=dt))

(x)' = -2.602 x +  0.428 v + -0.193 x^2 +  0.114 x v +  2.733 sin(1 x) +  0.282 sin(1 v)
(v)' = -0.758 x + -0.250 x^2 +  0.186 x v
Model score (R^2): 0.9677484539973241


We now see, that on the observed scope, SINDy finds the usage of sin(x) justified. It fits more closely, although from physics it seems that it shouldn't be the case (since we now the ODE describing damped string problem).

In [ ]:
# --- 1. SIMULATION ---
# We simulate the first 200 points for a quick performance check
num_steps = 50 
t_test = t[:num_steps]

print("Simulation in progress...")
# model1: Polynomial library only
x_sim_poly = model1.simulate(X[0], t_test)

# model2: Combined Polynomial + Fourier library
x_sim_combined = model2.simulate(X[0], t_test)
print("Simulation complete!")

# --- 2. TIME SERIES PLOT ---
plt.figure(figsize=(12, 6))

# Original Data (Target)
plt.plot(t_test, X[:num_steps, 0], 'k', alpha=0.6, lw=3, label='Original Data (Kaggle)')

# Model 1 reconstruction
plt.plot(t_test, x_sim_poly[:, 0], 'r--', lw=2, label=f'Polynomial Model (R²={model1.score(X, t=dt):.3f})')

# Model 2 reconstruction
plt.plot(t_test, x_sim_combined[:, 0], 'b:', lw=2, label=f'Poly + Fourier Model (R²={model2.score(X, t=dt):.3f})')

plt.title("SINDy Reconstruction: Model Comparison (First 200 steps)", fontsize=14)
plt.xlabel("Time [t]", fontsize=12)
plt.ylabel("Position [x]", fontsize=12)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.show()

# --- 3. PHASE PORTRAIT (Velocity vs Position) ---
plt.figure(figsize=(8, 8))

# Data trajectory
plt.plot(X[:num_steps, 0], X[:num_steps, 1], 'k', alpha=0.4, label='Ground Truth Trajectory')

# Model trajectories
plt.plot(x_sim_poly[:, 0], x_sim_poly[:, 1], 'r--', label='Polynomial Model Path')
plt.plot(x_sim_combined[:, 0], x_sim_combined[:, 1], 'b:', label='Fourier Model Path')

plt.xlabel('Position (x)', fontsize=12)
plt.ylabel('Velocity (v)', fontsize=12)
plt.title('Phase Portrait: System Stability Analysis', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()